Importando as bibliotecas utilizadas ... 

In [2]:
import numpy as np
import pandas as pd
import pickle
import gensim
import spacy

D:\ANACONDA\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Inicializando o DataFrame da matriz dos conhecimentos, 
utilizando a função 'fillna' para consertar as células mescladas,
lendo a partir da linha 4
e lendo até a coluna 9 (excluso)

In [3]:
crea_df = pd.read_excel("Matriz_do_Conhecimento.xls", skiprows=4).iloc[:, :9].fillna(method="ffill")

Corrigindo valores que foram incorretamente preenchido pelo método fillna("ffill"):

In [4]:
crea_df = crea_df.replace({"TÓPICOS": np.NaN, "Nº DE ORDEM DOS TÓPICOS": np.NaN})

Salvando o Dataframe com o pickle

In [5]:
with open('crea_df_pickle.txt', 'wb') as f:
    pickle.dump(crea_df, f)

Abrindo o dataFrame em formato Binário e Imprimindo o DataFrame para fins de visualização:

In [6]:
with open('crea_df_pickle.txt', 'rb') as f:
    crea_df = pickle.load(f)

# * subseção de Construção de Edificações
subsection1_df = crea_df[ crea_df['SUB-SETOR' ] == 'Construção de Edificações']

mechatronics_df = crea_df[ crea_df['SETOR'] == 'Controle e Automação']

# mechatronics_df

Importando as matérias que foram manualmente obtidas e salvas em um arquivo json

In [7]:
subjects_df = pd.read_json("MechatronicsEngeneeringSubjects.json")

# subjects_df

Criando uma função de pre-processamento de texto. Retira ruidos(cleaning) -> tokeniza-lemmatiza -> depois retira stopwords;

Recebe um texto no formato de string e retorna uma lista de strings com as palavras do documento.

In [8]:
from nltk.corpus import stopwords

nlp = spacy.load('pt_core_news_lg')

# TODO: find out if it is definately impossible to disable PoS tags in nlp model; alternative: convert all verbs to noun using wordnet and .pos from spacy?
# TODO: implementar o tagger utilizando o seguinte banco de dados: https://www.nltk.org/howto/portuguese_en.html#accessing-the-macmorpho-tagged-corpus
# tagger = nlp.get_pipe("tagger")
# doc = nlp("eletromagnetismo serie")
# print(tagger.model.predict([doc])[0][1])
# print(tagger.labels)

# * adding custom texts that dont represent real words
noises_list = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x", "xi"]

stopWords_list = stopwords.words("portuguese")

# * adding custom words to StopWords list
stopWords_list += [
    'referente',
    'seguinte',
    'etc',
    'ª',
    'tal',
    'um', 
    'dois',
    'tres',
    'vs',
    'aula',
    'tal',
]

# * preprocessing stopwords to correct format
stopWords_list = gensim.utils.simple_preprocess(" ".join(stopWords_list), deacc=True, min_len=1, max_len=40)

# print(stopWords_list)

# * manual intervention, changing final lemmas
intervention_dict = {
    "campar": "campo",
    "seriar":"serie",
    "eletromagnetico":"eletromagnetismo",
}

def preprocess(text):
    # * importing stopwords from nltk and spacy pipeline
    global nlp
    global stopWords_list
    global noises_list
    global intervention_dict

    # * preprocessing text with gensim.simple_preprocess, eliminating noises: lowercase, tokenized, no symbols, no numbers, no accents marks(normatize)
    text_list = gensim.utils.simple_preprocess(text, deacc=True, min_len=1, max_len=40)

    # * recombining tokens to a string type object and removing remaining noises
    text_str = " ".join([word for word in text_list if word not in noises_list])

    # * preprocessing with spacy, retokenizing -> tagging parts of speech (PoS) -> parsing (assigning dependencies between words) -> lemmatizing
    text_doc = nlp(text_str)

    # * re-tokenization, removing stopwords and lemmatizing
    lemmatized_text_list = [token.lemma_ for token in text_doc if token.text not in stopWords_list]

    # * manual intervention conversion of lemmas and removing 1 letter stopwords
    output = []
    for token in lemmatized_text_list:
        if len(token) <= 1:
            continue
        if token in intervention_dict:
            output.append(intervention_dict[token])
        else:
            output.append(token)
            
    return output

Pre-processando as matérias com Stopwords do NLTK e função do Gensim. E adicionando uma nova columa ao Dataframe que consiste em todo o texto da matéria

In [9]:
documents_list = []

for i, row in subjects_df.iterrows():

    # * reading values of each subject (row)
    subject_id = row["codigo"]
    name = row["nome"]
    syllabus = row["ementa"]
    content = row["conteudo"]

    # * combining them to create the subject document
    text = name + ' ' + syllabus + ' ' + content
    
    # * preprocessing
    preProcessedText = preprocess(text)
    documents_list.append(preProcessedText)

# print(documents_list)

documents_series = pd.Series(documents_list, name="documento")

documents_df = pd.concat([subjects_df, documents_series], axis=1)

Salvando o Dataframe das matérias,  reabrindo-o e imprimindo o DataFrame para fins de visualização:

In [10]:
with open('documents_df.txt', 'wb') as f:
    pickle.dump(documents_df, f)

In [11]:
with open('documents_df.txt', 'rb') as f:
    documents_df = pickle.load(f)

# documents_df

Salvando a coluna de documentos para tornar legível e facilitar a busca, assim podendo conferir se os lemmas estão satisfatórios e se condizem

In [12]:
import json
with open("debbuging_docs.json", "w+") as f:
    json.dump(documents_df['documento'].to_dict(), f, indent=4, ensure_ascii=True)

construindo o Corpus, vetorizando os documentos com o bag-of-words do gensim

In [13]:
from gensim import corpora

lemmatizedData = documents_df["documento"].tolist()

# * gensim dictionary object, which will track each word to its respective id
id2wordDict = corpora.Dictionary(lemmatizedData)

# * gensim doc2bow method to map each word to a integer id and its respective frequency
corpus = [id2wordDict.doc2bow(text) for text in lemmatizedData]

# * list of list of tuples (id of a word, frequency)
# corpus

Utilizando tf-idf do gensim para descobrir as palavras mais importantes e dar mais pesos a elas;

tupla(int, int) -> tupla(int, float)

td-idf(term_i, document_j) = freq(i,j) * log2 ( inverse_document_frequency(i) )

freq(i, j) = total occurances of i in j / total words in j

inverse_document_frequency(i) = total documents / documents that have at least one occurance of i 

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs.

In [14]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=id2wordDict)

LSI, pega a matriz do corpus e a decompõe utilizando o SVD. Das três matrizes criadas, utiliza-se apenas o right singular vectors que representa a relação entre os tópicos latentes com as palavras.

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs. E o parâmetro power_iters define o número de iterações para treinamento do modelo, e, portanto, quanto maior o valor, mais acurado e mais devagar vai ser o treino modelo

In [15]:
lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=id2wordDict, num_topics=100, power_iters=100)

# lsi_model.print_topics(num_topics=56)

left singular vector -> term-to-topic matrix (não será utilizado)

In [16]:
# * U Matrix
print(np.shape(lsi_model.projection.u))

# lsi_model.projection.u

(2030, 57)


singular values -> "impacto" (abrangência?) de cada tópico -> "feature importance"

Pode-se escolher um valor de corte para reduzir o valor do parametro num_topics, aumentando, assim, o desempenho de tempo. O número de tópicos latentes não tem como ser maior que o numero de documentos

In [17]:
print("número de documentos/ementas/matérias:", len(subjects_df))
print("topicos latentes encontrados com um \'impacto alto\':", len(lsi_model.projection.s))

# * S matrix (sigma)
# print(np.shape(lsi_model.projection.s))
# lsi_model.projection.s

número de documentos/ementas/matérias: 58
topicos latentes encontrados com um 'impacto alto': 57


right singular vectors -> document-to-topic matrix (Não é diretamente calculado e armazenado pois pode ser muito grande devido a quantidade de documentos -> num_topic x documents)

Será o documento armazenado e utilizado para fazer as queries de similaridade

In [18]:
V_matrix = gensim.matutils.corpus2dense(lsi_model[tfidf_model[corpus]], len(lsi_model.projection.s)).T / lsi_model.projection.s
print(np.shape(V_matrix))
# * V or V^T matrix
# * representado por lsi_model[x]
V_matrix

(58, 57)


array([[-8.16842379e-02, -1.90699447e-02,  3.26055870e-03, ...,
        -1.17165794e-02,  1.53363475e-02, -4.56397882e-03],
       [-1.87783694e-01,  1.58409031e-02, -3.87895189e-01, ...,
        -2.08063679e-02, -1.61471958e-02, -2.67343672e-02],
       [-1.26493891e-01,  1.25987436e-02, -2.75186795e-01, ...,
         1.61533114e-02, -5.41333898e-03,  7.17352750e-03],
       ...,
       [-6.52923932e-02, -3.26972188e-02, -3.83349938e-02, ...,
         9.92697377e-04,  3.13467224e-04, -5.76186915e-04],
       [-2.21970532e-02,  2.30684218e-03, -1.23568911e-02, ...,
         4.02028368e-03, -5.54040801e-04,  7.26040521e-04],
       [-1.34881176e-02, -1.68342006e-02, -2.67376240e-03, ...,
         1.11045832e-03, -1.30478494e-03,  3.83274140e-04]])

Salvando, então, o corpus já processado pelo tf_idf e também a matrix V de LSI em um arquivo texto do tipo Matrix Market format, que permite que a matriz seja guardado em um arquivo texto, mas também pode ser utilizado para cálculos. (o método Serialize guardas os indexes)

In [19]:
gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('lsi_model_mm', lsi_model[tfidf_model[corpus]])

Abrindo as matrizes salvas nos arquivos textos

In [20]:
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

# * features of td-idf matrix are the different words
print(tfidf_corpus)

# * features of LSI model are the latent topics discovered
print(lsi_corpus)

MmCorpus(58 documents, 2030 features, 4329 non-zero entries)
MmCorpus(58 documents, 57 features, 3306 non-zero entries)


Buscas de Similaridade Semântica utilizando o método Matrix similarity do gensim, o qual computa o "cosine similarity".

Se forem buscados termos que não foram adicionados ao dicionário id2wordDict, não há como mapear esse documento

O parâmetro num_best define quantos dos documentos mais proximos será buscado

In [21]:
from gensim.similarities.docsim import MatrixSimilarity

cosineSimilarity = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms, num_best=8)

def search_similarity_query(search_document):

    # * preprocessing and processing until becomes a matrix of type term_to_topic (V)
    doc = preprocess(search_document)
    query_bow = id2wordDict.doc2bow(doc)
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    # * cossine similarity between the vector of the new document vs all other vectors of documents
    # * returns a list of tuples (id of compared document, similarity)
    ranking = cosineSimilarity[query_lsi]

    ranking.sort(key=lambda unit: unit[1], reverse= True)
    result = []

    for subject in ranking:

        result.append (
            {
                'Relevancia': round((subject[1] * 100),6),
                'Código da Matéria': subjects_df['codigo'][subject[0]],
                'Nome da matéria': subjects_df['nome'][subject[0]]
            }

        )
    
    output = pd.DataFrame(result, columns=['Relevancia','Código da Matéria','Nome da matéria'])
    
    return output



Utilizando documentos no formato de str, pode-se realizar buscas de similaridade semântica entre as matérias

In [22]:
search_similarity_query("eletromagnético")
print(preprocess("eletromagnético"))

['eletromagnetismo']


Multi-queries:

In [27]:
print( mechatronics_df.shape )

query_full_text = ""
visited_topics = {}
for i in range(2211):
    text = mechatronics_df.iloc[i, 8] + ' ' + mechatronics_df.iloc[i, 7]
    if text in visited_topics:
        continue
    visited_topics[text] = (mechatronics_df.iloc[i, 8],  mechatronics_df.iloc[i, 7])
    # print(f'conteúdo buscado: {mechatronics_df.iloc[i, 8]},\nárea do conhecimento: {mechatronics_df.iloc[i, 7]}')
    # print(f'texto buscado depois de preprocessado: {preprocess(text)}')
    # print( search_similarity_query(text), end='\n\n')

    query_full_text += "Index: " + str(i) + '\n'
    query_full_text += "Conteúdo buscado: " + mechatronics_df.iloc[i, 8] + '\n'
    query_full_text += "área do conhecimento: " + mechatronics_df.iloc[i, 7] + '\n'
    query_full_text += "Texto buscado depois de preprocessado: " + str(preprocess(text)) + '\n'
    query_full_text += search_similarity_query(text).to_string() + "\n\n"

with open("mechatronics_requirements_query.txt", 'w') as f:
    f.write(query_full_text)

# print(len(visited_topics))
# for i in visited_topics.items():
#     print(i)


(2211, 9)


Writing all topics and syllabus into a xlsx file

In [39]:
import xlsxwriter

workbook = xlsxwriter.Workbook('conteudos_e_ementas.xlsx')

worksheet1 = workbook.add_worksheet()

worksheet1.write(1, 1, "Código da Matéria")
worksheet1.write(1, 2, "Nome da Matéria")
row = 2
    
for i, line in subjects_df.iterrows():
    worksheet1.write(row, 1, line["codigo"])
    worksheet1.write(row, 2, line["nome"])
    row += 1

worksheet1.write(1, 4, "Conteúdos")
worksheet1.write(1, 5, "Área do Conhecimento")
worksheet1.write(1, 6, "Ementa(s) Associadas")
row = 2
for item in visited_topics.values():
    worksheet1.write(row, 4, item[0])
    worksheet1.write(row, 5, item[1])
    row += 1

workbook.close()